# alpha

In [1]:
from itertools import combinations

In [2]:
import numpy as np
import pandas as pd
from zipline.research import returns

In [3]:
stocks = [str(i) for i in range(300001,300021)]

In [4]:
mu = returns(stocks, '2018-3-20','2018-5-31').mean()

[2018-06-05 03:00:12.086822] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-31 01:31:00+00:00
最后收盘时间: 2018-05-31 07:00:00+00:00


In [5]:
mu

特锐德(300001)    -0.001444
神州泰岳(300002)   -0.002906
乐普医疗(300003)    0.004536
南风股份(300004)   -0.013959
探路者(300005)    -0.000831
莱美药业(300006)   -0.001242
汉威科技(300007)   -0.001003
天海防务(300008)   -0.004395
安科生物(300009)    0.000766
立思辰(300010)     0.006260
鼎汉技术(300011)   -0.005505
华测检测(300012)    0.003423
新宁物流(300013)   -0.002562
亿纬锂能(300014)   -0.000749
爱尔眼科(300015)    0.004830
北陆药业(300016)    0.004430
网宿科技(300017)   -0.003557
中元股份(300018)   -0.002182
硅宝科技(300019)   -0.004257
银江股份(300020)   -0.001900
dtype: float64

# 初始权重

In [6]:
init_assets = mu.iloc[2:8]

In [7]:
w_data = np.random.randn(len(init_assets))

In [8]:
init_w = pd.Series(w_data.T / sum(w_data), index = init_assets.index)

In [9]:
init_w

乐普医疗(300003)    0.404753
南风股份(300004)   -1.104706
探路者(300005)     1.083933
莱美药业(300006)   -2.098114
汉威科技(300007)    3.449065
天海防务(300008)   -0.734932
dtype: float64

# 优化

In [10]:
from zipline.optimize.objectives import TargetWeights, MaximizeAlpha
from zipline.optimize.constraints import (MaxGrossExposure, NetExposure,
                                          NotExceed, NotLessThan,
                                          DollarNeutral, FixedWeight)
from zipline.optimize.core import run_optimization, calculate_new_weights

## 最大alpha

In [11]:
obj = MaximizeAlpha(mu)

In [12]:
me = MaxGrossExposure(1.5)
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2],0.3)

In [13]:
calculate_new_weights(obj, [me,ne,fw], init_w)

特锐德(300001)    -0.0
神州泰岳(300002)   -0.0
乐普医疗(300003)    0.3
南风股份(300004)   -0.4
探路者(300005)    -0.0
莱美药业(300006)   -0.0
汉威科技(300007)   -0.0
天海防务(300008)   -0.0
安科生物(300009)    0.0
立思辰(300010)     0.4
鼎汉技术(300011)   -0.4
华测检测(300012)    0.0
新宁物流(300013)   -0.0
亿纬锂能(300014)   -0.0
爱尔眼科(300015)    0.0
北陆药业(300016)    0.0
网宿科技(300017)   -0.0
中元股份(300018)   -0.0
硅宝科技(300019)   -0.0
银江股份(300020)   -0.0
dtype: float64

## 目标权重

In [14]:
target_weights = pd.Series([0.1,-0.3,-0.2,0.7],index=mu.index[14:18])

In [15]:
target_weights

爱尔眼科(300015)    0.1
北陆药业(300016)   -0.3
网宿科技(300017)   -0.2
中元股份(300018)    0.7
dtype: float64

In [16]:
obj = TargetWeights(target_weights)

In [17]:
calculate_new_weights(obj, [me,ne], init_w)

乐普医疗(300003)    0.0
南风股份(300004)    0.0
探路者(300005)     0.0
莱美药业(300006)    0.0
汉威科技(300007)    0.0
天海防务(300008)    0.0
爱尔眼科(300015)    0.1
北陆药业(300016)   -0.3
网宿科技(300017)   -0.2
中元股份(300018)    0.4
dtype: float64

In [18]:
from toolz import concat
import cvxpy as cvx

## 可解情形

In [19]:
objective = MaximizeAlpha(mu)
me = MaxGrossExposure(1.5)
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2],0.3)
constraints = [me,ne]

In [20]:
cvx_objective = objective.to_cvxpy(init_w)
new_weights = objective.new_weights
new_weights_series = objective.new_weights_series
constraint_map = {
    c: c.to_cvxpy(new_weights, new_weights_series, init_w)
    for c in constraints
}
cvx_constraints = list(concat(constraint_map.values()))

In [21]:
problem = cvx.Problem(cvx_objective, cvx_constraints)
problem.solve()

0.01173867217044785

In [22]:
new_weights.value

array([-7.85244334e-10, -1.93639781e-09,  1.32003964e-09, -3.99999999e-01,
       -4.32292255e-10, -6.63800408e-10, -5.27224512e-10, -2.96415058e-09,
        3.97299800e-10,  3.99999992e-01, -3.99999986e-01,  2.49847970e-09,
       -1.60897951e-09, -3.88228080e-10,  2.99999962e-01,  2.84048279e-09,
       -2.64927256e-09, -1.29377376e-09, -3.10403912e-09, -1.08524316e-09])

In [23]:
problem.solver_stats.solver_name

'ECOS'

# 无解问题

## 冲突限制

In [24]:
objective = MaximizeAlpha(mu)
me = MaxGrossExposure(1.5)
dn = DollarNeutral(0.001)
nl = NotLessThan(0.5)  # 单个权重不少于0.5与每个权重不得超过0.4矛盾
ne = NotExceed(0.4)
fw = FixedWeight(mu.index[2], 0.3)
constraints = [me, dn, ne, nl, fw]

In [25]:
cvx_objective = objective.to_cvxpy(init_w)
new_weights = objective.new_weights
new_weights_series = objective.new_weights_series
constraint_map = {
    c: c.to_cvxpy(new_weights, new_weights_series, init_w)
    for c in constraints
}
cvx_constraints = list(concat(constraint_map.values()))

In [26]:
infeasible_problem = cvx.Problem(cvx_objective, cvx_constraints)
infeasible_problem.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -9.682e-04  -7.519e+00  +2e+02  7e-01  6e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  +4.133e-03  -1.651e+00  +4e+01  7e-01  2e-01  2e+00  4e-01  0.7997  9e-02   1  0  0 |  0  0
 2  +1.276e-03  +8.561e+01  +3e+00  6e+00  6e-01  1e+02  3e-02  0.9890  5e-02   1  0  0 |  0  0
 3  +1.355e-03  +8.102e+03  +3e-02  6e+00  6e-01  8e+03  3e-04  0.9890  1e-04   1  0  0 |  0  0
 4  +1.390e-03  +7.206e+05  +4e-04  6e+00  6e-01  7e+05  3e-06  0.9890  1e-04   2  0  0 |  0  0
 5  +1.424e-03  +6.404e+07  +4e-06  6e+00  6e-01  6e+07  4e-08  0.9890  1e-04   1  0  0 |  0  0
 6  +1.463e-03  +5.705e+09  +4e-08  6e+00  6e-01  6e+09  4e-10  0.9890  1e-04   0  0  0 |  0  0

PRIMAL INFEASIBLE (within feastol=1.2e-10).
Runtime: 0.000535 seconds.



-inf

In [27]:
infeasible_problem.status

'infeasible'

## 解析限定范围

### 限定对象

### 限定对象的常量

In [28]:
import re

In [29]:
NUM_SIGN_PATTERN = re.compile(r'(-)\d{1,}.\d{1,}')
VAR_SIGN_PATTERN = re.compile(r'(-)new_weights')
INVALID_PATTERN = re.compile(r'\((new_weights)')
NUM_PATTERN = re.compile(r'[-]?(\d{1,}.\d{1,})')
VAR_ID_PATTERN = re.compile(r'.*?\[(\d{1,})\]')

In [30]:
def search_v(expr, pat):
    s = re.search(pat, expr)
    if s is None:
        return None
    else:
        return s.groups()[0]

In [31]:
def parse_constraint_range(constraint):
    """解析单个限制其权重范围信息"""
    min_, max_ = -np.inf, np.inf
    expr = str(constraint)
    # 忽略
    invalid = search_v(expr, INVALID_PATTERN)
    if invalid:
        invalid = True
    else:
        invalid = False
    # 常量符号
    c_sign = search_v(expr, NUM_SIGN_PATTERN)
    if c_sign is None:
        c_sign = 1
    else:
        c_sign = -1
    # 变量符号
    v_sign = search_v(expr, VAR_SIGN_PATTERN)
    if v_sign is None:
        v_sign = 1
    else:
        v_sign = -1
    
    # 系数值
    c_num = search_v(expr, NUM_PATTERN)
    if c_num is not None:
        c_num = c_sign * float(c_num)
        
    # 变量切片序号
    id_num = search_v(expr, VAR_ID_PATTERN)
    if id_num is not None:
        id_num =  int(id_num)
    
    if invalid:
        return (None, None, None)
    else:
        if isinstance(constraint, cvx.Zero):
            min_ = -c_num #* c_sign #* v_sign
            max_ = -c_num #* c_sign #* v_sign
        elif isinstance(constraint, cvx.NonPos):
            if v_sign == -1:
                min_ = c_num
            else:
                max_ = -c_num
    return (id_num, min_, max_)

In [32]:
def gen_ranges(n, rngs):
    res = {k: [-np.inf, np.inf] for k in range(n)}
    for r in rngs:
        if r[0] is not None:
            # 更新下限
            old = res[r[0]][0]
            new = r[1]
            res[r[0]][0] = max(old, new)
            # 更新上限
            old = res[r[0]][1]
            new = r[2]
            res[r[0]][1] = min(old, new)
        else:
            for i in range(n):
                res[i][0] = -np.inf if r[1] is None else r[1]
                res[i][1] = np.inf if r[2] is None else r[2]
    return res

In [33]:
def check_violate(limit_a, limit_b):
    """如不相交，违背"""
    if (limit_a[0] > limit_b[1]) or (limit_b[0] > limit_a[1]):
        return True
    return False

In [34]:
def run_diagnostics(objective, new_weights, cvx_objective, constraint_map):
    info = '没有找到满足所有必需约束的投资组合，尝试优化失败。'
    info += '检查以下投资组合，发现违背约束条件：\n'
    asstes = objective.new_weights_series.index
    diagnostics = {}
    n = new_weights.size
    for k, cons in constraint_map.items():
        rngs = []
        for c in cons:
            rngs.append(parse_constraint_range(c))
        diagnostics[k] = gen_ranges(n, rngs)
    for k1, k2 in combinations(diagnostics.keys(), 2):
        limit_as = diagnostics[k1]
        limit_bs = diagnostics[k2]
        for i, (limit_a, limit_b) in enumerate(
                zip(limit_as.values(), limit_bs.values())):
            if check_violate(limit_a, limit_b):
                asset = asstes[i]
                #msg  = '{}不可能同时满足{}和{}约束'.format(asset，k1, k2)
                msg = '{}不可能同时满足{}和{}\n'.format(asset, k1, k2)
                info += msg
    return info

In [35]:
print(run_diagnostics(objective, new_weights, cvx_objective, constraint_map))

没有找到满足所有必需约束的投资组合，尝试优化失败。检查以下投资组合，发现违背约束条件：
特锐德(300001)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
神州泰岳(300002)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
乐普医疗(300003)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
南风股份(300004)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
探路者(300005)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
莱美药业(300006)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
汉威科技(300007)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
天海防务(300008)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
安科生物(300009)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
立思辰(300010)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
鼎汉技术(300011)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
华测检测(300012)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
新宁物流(300013)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
亿纬锂能(300014)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLessThan(权重>=0.5)
爱尔眼科(300015)不可能同时满足NotExceed(区间[-0.4,+0.4])和NotLe